In [ ]:
using WAV;
using Gen;
import Random
include("model.jl")
include("plotting.jl")
include("proposals.jl")
include("gammatonegram.jl")
include("time_helpers.jl")

## Switch involution examples

Basic model settings

In [ ]:
duration = 0.81
max_tones = 10
audio_sr = 20000
tstep = 0.020
mindur = 0.020
wts, f = gtg_weights(audio_sr);

# function get_kernel_params(var_type) 
#     if var_type == :erb
#         sigma = 3
#         scale = 0.5
#         noise = 1
#     elseif var_type == :amp
#         sigma = 1
#         scale = 0.5
#         noise = 0.5
#     end
#     return sigma, scale, noise    
# end

### Two-tone proposal example

In [ ]:
difference = -1

constraints = choicemap()

constraints[:n_sources] = 2

constraints[:source => 1 => :n_tones] = 1
mu_erb_1 = 8; mu_amp_1 =0.9;
constraints[:source => 1 => :mu_erb] = mu_erb_1
constraints[:source => 1 => :mu_amp] = mu_amp_1
wait_1 = 0.101; dur_minus_min_1 = 0.080
constraints[:source => 1 => (:tone, 1) => :wait] = wait_1
constraints[:source => 1 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t1 = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + mindur], tstep)
constraints[:source => 1 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t1))
constraints[:source => 1 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t1))

constraints[:source => 2 => :n_tones] = 1
mu_erb_2 = mu_erb_1 + difference; 
constraints[:source => 2 => :mu_erb] = mu_erb_2
constraints[:source => 2 => :mu_amp] = mu_amp_1
wait_1 = 0.3201; dur_minus_min_1 = 0.080
constraints[:source => 2 => (:tone, 1) => :wait] = wait_1
constraints[:source => 2 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t2 = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + mindur], tstep)
constraints[:source => 2 => (:tone, 1) => :erb]= fill(mu_erb_2, length(t2))
constraints[:source => 2 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t2))

t_full = vcat(t1, t2)
erb_full = vcat(constraints[:source => 1 => (:tone, 1) => :erb], constraints[:source => 2 => (:tone, 1) => :erb])

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 

In [ ]:
function get_initial_trace(observation_trace, constraints)
    constraints[:scene] = observation_trace[:scene]
    constraints[:noise] = 1.0
    initial_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
    plot_sources(initial_trace, observation_trace[:scene], 0)
    return initial_trace
end
initial_trace = get_initial_trace(observation_trace, constraints);

In [ ]:
function make_switch_proposal(initial_trace, observation_trace, show_info)
    (fwd_choices, fwd_score, fwd_ret) = propose(switch_randomness, (initial_trace,))
    (new_trace, bwd_choices, weight) = switch_involution(initial_trace, fwd_choices, fwd_ret, ())
    (bwd_score, _) = assess(switch_randomness, (new_trace,), bwd_choices)
    alpha = weight - fwd_score + bwd_score
    if show_info
        println((weight - fwd_score + bwd_score))
        println("Weight: $(weight), Fwd: $(fwd_score), Bwd: $(bwd_score)")
        plot_sources(new_trace, observation_trace[:scene], 1)
        println(fwd_choices)
        println("Backward choices")
        println(bwd_choices)
    end
    return new_trace, alpha
end
new_trace, alpha = make_switch_proposal(initial_trace, observation_trace, true);

In [ ]:
function check_score(initial_trace, new_trace, observation_trace)
    println("Looking at each part of the score:");println("")
    @assert initial_trace[:noise] == new_trace[:noise]
    noise_value = initial_trace[:noise]
    # old_scene, _, _ = get_retval(initial_trace)
    # new_scene, _, _ = get_retval(new_trace)
    ## NOTE: if you use "old_scene" or "new_scene" instead of the value from the trace, the likelihood does NOT come out equal!!
    old_likelihood=logpdf(noisy_matrix,initial_trace[:scene],observation_trace[:scene],noise_value)
    new_likelihood=logpdf(noisy_matrix,new_trace[:scene],observation_trace[:scene],noise_value)
    println("Old likelihood: $old_likelihood, New likelihood: $new_likelihood")
    @assert old_likelihood == new_likelihood
    println("Likelihoods are equal.")

    println(""); println("Score of random choices:")
    for trace_number = 1:2
        trace = trace_number == 1 ? initial_trace : new_trace
        trace_name = trace_number == 1 ? "Initial" : "New"
        
        total = 0
        for i = 1:trace[:n_sources]
            single = project(trace, select(:source => i))
            total += single
            println("$trace_name Trace -- source $i: $single")
            mu_erb = round(trace[:source => i => :mu_erb], digits=2)
            println("\tmu erb $mu_erb: ", project(trace, select(:source => i => :mu_erb)))
            mu_amp = round(trace[:source => i => :mu_amp], digits=2)
            println("\tmu amp $mu_amp: ", project(trace, select(:source => i => :mu_amp)))
            n_tones = trace[:source => i => :n_tones]
            println("\tn_tones $n_tones: ", project(trace, select(:source => i => :n_tones)))
            for j = 1:trace[:source => i => :n_tones]
                println("\tTone $j")
                for a = [:wait, :dur_minus_min, :erb, :amp]
#                     println(trace[:source => i => (:tone, j) => a])
                    println("\t\t", a, ": ", project(trace, select(:source => i => (:tone, j) => a)))
                end
            end
        end
        println("$trace_name Trace -- total: $total");println("")
            
    end

end
check_score(initial_trace, new_trace, observation_trace)

In [ ]:
println("Checking the correctness of the conditional Gaussian distributions:")
mu, cov =  get_mu_cov(t_full, new_trace[:source => 1 => :mu_erb], :erb)
full_cov_score=logpdf(mvnormal,erb_full,mu,cov)
println("full_cov_score: $full_cov_score")
tone1_erb_score = project(new_trace, select(:source => 1 => (:tone, 1) => :erb))
tone2_erb_score = project(new_trace, select(:source => 1 => (:tone, 2) => :erb))
println("conditional score: ", tone1_erb_score + tone2_erb_score)

### Three-tone proposal example

In [ ]:
stream_difference = 20 #between the tones with the same timing
switch_difference = 1 #between source_1 tone, and the switch_tone

constraints = choicemap()

constraints[:n_sources] = 2

constraints[:source => 1 => :n_tones] = 1
mu_erb_1 = 8; mu_amp_1 =0.9;
constraints[:source => 1 => :mu_erb] = mu_erb_1
constraints[:source => 1 => :mu_amp] = mu_amp_1
wait_1 = 0.101; dur_minus_min_1 = 0.080
constraints[:source => 1 => (:tone, 1) => :wait] = wait_1
constraints[:source => 1 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 1 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t))
constraints[:source => 1 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))


constraints[:source => 2 => :n_tones] = 2
tone_erb_1 = mu_erb_1 + stream_difference; mu_amp_1 =0.9;
wait_1 = 0.101; dur_minus_min_1 = 0.080
constraints[:source => 2 => (:tone, 1) => :wait] = wait_1
constraints[:source => 2 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 2 => (:tone, 1) => :erb]= fill(tone_erb_1, length(t))
constraints[:source => 2 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))


tone_erb_2 = mu_erb_1 + switch_difference 
wait_2 = 0.101; 
constraints[:source => 2 => (:tone, 2) => :wait] = wait_2
constraints[:source => 2 => (:tone, 2) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_2, wait_2+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 2 => (:tone, 2) => :erb]= fill(tone_erb_2, length(t))
constraints[:source => 2 => (:tone, 2) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 2 => :mu_amp] = mu_amp_1
constraints[:source => 2 => :mu_erb] = (tone_erb_1 + tone_erb_2)/2

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 

In [ ]:
initial_trace = get_initial_trace(observation_trace, constraints);

In [ ]:
new_trace, alpha = make_switch_proposal(initial_trace, observation_trace, true);

In [ ]:
check_score(initial_trace, new_trace, observation_trace)

In [ ]:
function run_switch_inference(initial_trace, observation_trace, n_steps)
    trace = initial_trace
    trace_list = []
    proposal_list = []
    alpha_list = []
    for i = 1:n_steps
        new_trace, alpha = make_switch_proposal(trace, observation_trace, false);
        push!(proposal_list, new_trace)
        f = log(rand())
        accept = f < alpha
        trace = accept ? new_trace : trace
        push!(trace_list, trace)
        push!(alpha_list, alpha)
    end
    return trace_list, proposal_list, alpha_list
end
trace_list, proposal_list, alpha_list = run_switch_inference(initial_trace, observation_trace, 100);

In [ ]:
function plot_trace(trace_list)
    for i = 1:length(trace_list)
        plot_sources(trace_list[i], observation_trace[:scene], i)
    end
end
function plot_proposals(trace_list, proposal_list)
    for i = 1:length(trace_list)
        plot_sources(trace_list[i], observation_trace[:scene], i)
        plot_sources(proposal_list[i], observation_trace[:scene], i,colors="Reds")
    end
end
#plot_trace(trace_list)
plot_proposals(trace_list, proposal_list)

### 6-tone inference example with repetition

In [ ]:
stream_difference = 8

constraints = choicemap()

constraints[:n_sources] = 6

constraints[:source => 1 => :n_tones] = 1
mu_erb_1 = 12; mu_amp_1 =0.9;
constraints[:source => 1 => :mu_erb] = mu_erb_1
constraints[:source => 1 => :mu_amp] = mu_amp_1
wait_1 = 0.101; dur_minus_min_1 = 0.080
constraints[:source => 1 => (:tone, 1) => :wait] = wait_1
constraints[:source => 1 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 1 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t))
constraints[:source => 1 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 2 => :n_tones] = 1
mu_erb_2 = mu_erb_1 + stream_difference; mu_amp_1 =0.9;
constraints[:source => 2 => :mu_erb] = mu_erb_2
constraints[:source => 2 => :mu_amp] = mu_amp_1
wait_1 = 0.2101; dur_minus_min_1 = 0.080
constraints[:source => 2 => (:tone, 1) => :wait] = wait_1
constraints[:source => 2 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 2 => (:tone, 1) => :erb]= fill(mu_erb_2, length(t))
constraints[:source => 2 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 3 => :n_tones] = 1
mu_amp_1 =0.9;
constraints[:source => 3 => :mu_erb] = mu_erb_1
constraints[:source => 3 => :mu_amp] = mu_amp_1
wait_1 = 0.3201; dur_minus_min_1 = 0.080
constraints[:source => 3 => (:tone, 1) => :wait] = wait_1
constraints[:source => 3 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 3 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t))
constraints[:source => 3 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 4 => :n_tones] = 1
 mu_amp_1 =0.9;
constraints[:source => 4 => :mu_erb] = mu_erb_2
constraints[:source => 4 => :mu_amp] = mu_amp_1
wait_1 = 0.4301; dur_minus_min_1 = 0.080
constraints[:source => 4 => (:tone, 1) => :wait] = wait_1
constraints[:source => 4 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 4 => (:tone, 1) => :erb]= fill(mu_erb_2, length(t))
constraints[:source => 4 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 5 => :n_tones] = 1
mu_amp_1 =0.9;
constraints[:source => 5 => :mu_erb] = mu_erb_1
constraints[:source => 5 => :mu_amp] = mu_amp_1
wait_1 = 0.5401; dur_minus_min_1 = 0.080
constraints[:source => 5 => (:tone, 1) => :wait] = wait_1
constraints[:source => 5 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 5 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t))
constraints[:source => 5 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 6 => :n_tones] = 1
 mu_amp_1 =0.9;
constraints[:source => 6 => :mu_erb] = mu_erb_2
constraints[:source => 6 => :mu_amp] = mu_amp_1
wait_1 = 0.6501; dur_minus_min_1 = 0.080
constraints[:source => 6 => (:tone, 1) => :wait] = wait_1
constraints[:source => 6 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 6 => (:tone, 1) => :erb]= fill(mu_erb_2, length(t))
constraints[:source => 6 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 

In [ ]:
initial_trace = get_initial_trace(observation_trace, constraints);

In [ ]:
new_trace, alpha = make_switch_proposal(initial_trace, observation_trace, true);

In [ ]:
check_score(initial_trace, new_trace, observation_trace)

In [ ]:
trace_list, proposal_list, alpha_list = run_switch_inference(initial_trace, observation_trace, 100);

In [ ]:
plot_proposals(trace_list, proposal_list)

In [ ]:
new_trace,alpha = make_switch_proposal(trace_list[end], observation_trace,true);

In [ ]:
check_score(trace_list[end], new_trace, observation_trace)

### 6-tone example with variance

In [ ]:
stream_difference = 15
up_difference = 1
down_difference = 1

constraints = choicemap()

constraints[:n_sources] = 6

constraints[:source => 1 => :n_tones] = 1
mu_erb_1 = 7; mu_amp_1 =0.9;
constraints[:source => 1 => :mu_erb] = mu_erb_1
constraints[:source => 1 => :mu_amp] = mu_amp_1
wait_1 = 0.101; dur_minus_min_1 = 0.080
constraints[:source => 1 => (:tone, 1) => :wait] = wait_1
constraints[:source => 1 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 1 => (:tone, 1) => :erb]= fill(mu_erb_1, length(t))
constraints[:source => 1 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 2 => :n_tones] = 1
mu_erb_2 = mu_erb_1 + stream_difference; mu_amp_1 =0.9;
constraints[:source => 2 => :mu_erb] = mu_erb_2
constraints[:source => 2 => :mu_amp] = mu_amp_1
wait_1 = 0.2101; dur_minus_min_1 = 0.080
constraints[:source => 2 => (:tone, 1) => :wait] = wait_1
constraints[:source => 2 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 2 => (:tone, 1) => :erb]= fill(mu_erb_2, length(t))
constraints[:source => 2 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 3 => :n_tones] = 1
mu_amp_1 =0.9;
constraints[:source => 3 => :mu_erb] = mu_erb_1 + up_difference
constraints[:source => 3 => :mu_amp] = mu_amp_1
wait_1 = 0.3201; dur_minus_min_1 = 0.080
constraints[:source => 3 => (:tone, 1) => :wait] = wait_1
constraints[:source => 3 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 3 => (:tone, 1) => :erb]= fill(mu_erb_1 + up_difference, length(t))
constraints[:source => 3 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 4 => :n_tones] = 1
 mu_amp_1 =0.9;
constraints[:source => 4 => :mu_erb] = mu_erb_2 + up_difference
constraints[:source => 4 => :mu_amp] = mu_amp_1
wait_1 = 0.4301; dur_minus_min_1 = 0.080
constraints[:source => 4 => (:tone, 1) => :wait] = wait_1
constraints[:source => 4 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 4 => (:tone, 1) => :erb]= fill(mu_erb_2 + up_difference, length(t))
constraints[:source => 4 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 5 => :n_tones] = 1
mu_amp_1 =0.9;
constraints[:source => 5 => :mu_erb] = mu_erb_1 - down_difference
constraints[:source => 5 => :mu_amp] = mu_amp_1
wait_1 = 0.5401; dur_minus_min_1 = 0.080
constraints[:source => 5 => (:tone, 1) => :wait] = wait_1
constraints[:source => 5 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 5 => (:tone, 1) => :erb]= fill(mu_erb_1 - down_difference, length(t))
constraints[:source => 5 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:source => 6 => :n_tones] = 1
 mu_amp_1 =0.9;
constraints[:source => 6 => :mu_erb] = mu_erb_2 - down_difference
constraints[:source => 6 => :mu_amp] = mu_amp_1
wait_1 = 0.6501; dur_minus_min_1 = 0.080
constraints[:source => 6 => (:tone, 1) => :wait] = wait_1
constraints[:source => 6 => (:tone, 1) => :dur_minus_min] = dur_minus_min_1
t = get_tone_sample_times([wait_1, wait_1+dur_minus_min_1 + 0.020], tstep)
constraints[:source => 6 => (:tone, 1) => :erb]= fill(mu_erb_2 - down_difference, length(t))
constraints[:source => 6 => (:tone, 1) => :amp]= fill(mu_amp_1, length(t))

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 

In [ ]:
initial_trace = get_initial_trace(observation_trace, constraints);

In [ ]:
trace_list, proposal_list, alpha_list = run_switch_inference(initial_trace, observation_trace, 100);

In [ ]:
plot_proposals(trace_list, proposal_list)

## Real demo

In [ ]:
duration = 1.6

constraints = choicemap()

tone_frequencies = [1600, 400, 1270, 504, 1008, 635, 800, 800, 635, 1008,  504, 1270, 400, 1600]
tone_ERBs = freq_to_ERB.(tone_frequencies)
waits = [0.101, 0.201, 0.301, 0.401, 0.501, 0.601, 0.701, 0.801, 0.901, 1.001,  1.101, 1.201, 1.301, 1.401]

constraints[:n_sources] = length(tone_ERBs)

for j = 1:constraints[:n_sources]
    constraints[:source => j => :n_tones] = 1
    mu_erb = tone_ERBs[j]; mu_amp =0.9;
    constraints[:source => j => :mu_erb] = mu_erb
    constraints[:source => j => :mu_amp] = mu_amp
    wait = waits[j]; dur_minus_min = 0.080
    constraints[:source => j => (:tone, 1) => :wait] = wait
    constraints[:source => j => (:tone, 1) => :dur_minus_min] = dur_minus_min
    t = get_tone_sample_times([wait, wait+dur_minus_min + 0.020], tstep)
    constraints[:source => j => (:tone, 1) => :erb]= fill(mu_erb, length(t))
    constraints[:source => j => (:tone, 1) => :amp]= fill(mu_amp, length(t))
end

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 

In [ ]:
constraints = choicemap()

tone_frequencies = [1600, 400, 1270, 504, 1008, 635, 800, 800, 635, 1008,  504, 1270, 400, 1600]
tone_ERBs = freq_to_ERB.(tone_frequencies)
waits = [0.101, 0.201, 0.301, 0.401, 0.501, 0.601, 0.701, 0.801, 0.901, 1.001,  1.101, 1.201, 1.301, 1.401]

constraints[:n_sources] = 7
n = 1
for j = 1:2:length(tone_ERBs)
    constraints[:source => n => :n_tones] = 2
    mu_erb = tone_ERBs[j]; mu_amp =0.9;
    constraints[:source => n => :mu_erb] = mu_erb
    constraints[:source => n => :mu_amp] = mu_amp
    wait = waits[j]; dur_minus_min = 0.080
    constraints[:source => n => (:tone, 1) => :wait] = wait
    constraints[:source => n => (:tone, 1) => :dur_minus_min] = dur_minus_min
    t = get_tone_sample_times([wait, wait+dur_minus_min + 0.020], tstep)
    constraints[:source => n => (:tone, 1) => :erb]= fill(mu_erb, length(t))
    constraints[:source => n => (:tone, 1) => :amp]= fill(mu_amp, length(t))
    
    wait = 0.00001; dur_minus_min = 0.080
    constraints[:source => n => (:tone, 2) => :wait] = wait
    constraints[:source => n => (:tone, 2) => :dur_minus_min] = dur_minus_min
    t = get_tone_sample_times([wait, wait+dur_minus_min + 0.020], tstep)
    constraints[:source => n => (:tone, 2) => :erb]= fill(tone_ERBs[j+1], length(t))
    constraints[:source => n => (:tone, 2) => :amp]= fill(mu_amp, length(t))
    
    n = n + 1
end

constraints[:noise] = 0.5

observation_trace, _ = generate(generate_scene, (max_tones, duration, wts, mindur, tstep, audio_sr), constraints)
plot_gtg(observation_trace[:scene], duration, audio_sr/2) 
initial_trace = get_initial_trace(observation_trace, constraints);

In [ ]:
trace_list, proposal_list, alpha_list = run_switch_inference(initial_trace, observation_trace, 300);

In [ ]:
plot_proposals(trace_list[end-2:end], proposal_list[end-2:end])